In [12]:
import psycopg2

DB_NAME = "paier"
DB_USER = "postgres"
DB_PASSWORD = "1"
DB_HOST = "localhost"
DB_PORT = "5432"

# Get unique location of cabang PT SPILL
def get_unique_nama_lokasi():
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )

        cur = conn.cursor()
        query = "SELECT DISTINCT nama_lokasi FROM public.lokasi;"
        cur.execute(query)
        unique_locations = cur.fetchall()
        
        cur.close()
        conn.close()

        return [location[0] for location in unique_locations]

    except Exception as e:
        print(f"Error: {e}")
        return []

In [13]:
unique_locations = get_unique_nama_lokasi()
unique_locations

['DEPO PALARAN',
 'PKS 7',
 'TELUK BAYUR',
 'DEPO TARAKAN',
 'TPIL IX',
 'DEPO BALIKPAPAN',
 'DEPO PKS',
 'DEPO 4',
 'BATULICIN',
 'DEPO YONIF',
 'DEPO LINKAR',
 'DEPO TL 11',
 'BKA KALIANAK',
 'MEDAN',
 'DEPO JAPFA',
 'TAMBAK LANGON 11',
 'DEPO TB 4',
 'PERAK BARAT',
 'DEPO JAPFA PB',
 'TAMBAK LANGON 5',
 'DEPO TRISAKTI',
 'PONTIANAK']

# Prepare the Dataset

In [14]:
import pandas as pd
import requests
from datetime import datetime

# API for "Hari Besar", used for BKO recomendation
url = "https://dayoffapi.vercel.app/api"

response = requests.get(url)
data = response.json()

locations = unique_locations

current_year = datetime.now().year

# Find the keyword/unique value in API endpoint to get the event
event_mapping = [
    { "keyword": "masehi", "event": "Tahun Baru" },
    { "keyword": "imlek", "event": "Tahun Baru Imlek" }, 
    { "keyword": "nyepi", "event": "Hari Raya Nyepi" },
    { "keyword": "idul fitri", "event": "Hari Raya Idul Fitri" }, 
    { "keyword": "waisak", "event": "Hari Raya Waisak" },
    { "keyword": "idul adha", "event": "Hari Raya Idul Adha" },
    { "keyword": "kemerdekaan", "event": "Hari Kemerdekaan Republik Indonesia" },
    { "keyword": "natal", "event": "Hari Raya Natal" }
]

# Helper function to find the event in the mapping
def find_event(holiday_description):
    for event in event_mapping:
        if event["keyword"].lower() in holiday_description.lower():
            return event["event"], event["keyword"]
    return None, None

ConnectionError: HTTPSConnectionPool(host='dayoffapi.vercel.app', port=443): Max retries exceeded with url: /api (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001A5B9C43D90>: Failed to resolve 'dayoffapi.vercel.app' ([Errno 11001] getaddrinfo failed)"))

In [16]:
# Initialize a list to store the data
rows = []

# Group the events by name to manage shifts
event_dates = {}
for holiday in data:
    event_name, keyword = find_event(holiday["keterangan"])
    if event_name:
        if event_name not in event_dates:
            event_dates[event_name] = []
        event_dates[event_name].append((holiday["tanggal"], keyword))

# Iterate over the events and locations to populate the list
for event_name, dates in event_dates.items():
    for date_idx, (date, keyword) in enumerate(dates):
        shift = 1 if date_idx == 0 else 2
        for location in locations:
            rows.append({
                'Tanggal': date,
                'Hari': 1,
                'Lokasi': location,
                'Shift': shift,
                'Jumlah Personil': 1,
                'Harga Satuan': 260000,
                'Satuan': 'Orang/Shift',
                'Event': event_name,
                'Personil': 'TNI',
                'keyword': keyword
            })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(rows)

# Ensure 'Tanggal' is in datetime format
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

# Sort the DataFrame by 'Lokasi' and 'Tanggal'
df = df.sort_values(by=['Lokasi', 'Tanggal'])

# Reset index after sorting
df.reset_index(drop=True, inplace=True)

# Update 'Hari' column based on sorted DataFrame
for i in range(1, len(df)):
    if df.loc[i, 'Lokasi'] == df.loc[i-1, 'Lokasi'] and df.loc[i, 'keyword'] == df.loc[i-1, 'keyword']:
        df.loc[i, 'Hari'] = df.loc[i-1, 'Hari'] + 1
    else:
        df.loc[i, 'Hari'] = 1

df

,Tanggal,Hari,Lokasi,Shift,Jumlah Personil,Harga Satuan,Satuan,Event,Personil,keyword
0,2024-01-01,1,BATULICIN,1,1,260000,Orang/Shift,Tahun Baru,TNI,masehi
1,2024-02-09,1,BATULICIN,1,1,260000,Orang/Shift,Tahun Baru Imlek,TNI,imlek
2,2024-02-10,2,BATULICIN,2,1,260000,Orang/Shift,Tahun Baru Imlek,TNI,imlek
3,2024-03-11,1,BATULICIN,1,1,260000,Orang/Shift,Hari Raya Nyepi,TNI,nyepi
4,2024-03-12,2,BATULICIN,2,1,260000,Orang/Shift,Hari Raya Nyepi,TNI,nyepi
...,...,...,...,...,...,...,...,...,...,...
391,2024-06-17,1,TPIL IX,1,1,260000,Orang/Shift,Hari Raya Idul Adha,TNI,idul adha
392,2024-06-18,2,TPIL IX,2,1,260000,Orang/Shift,Hari Raya Idul Adha,TNI,idul adha
393,2024-08-17,1,TPIL IX,1,1,260000,Orang/Shift,Hari Kemerdekaan Republik Indonesia,TNI,kemerdekaan
394,2024-12-25,1,TPIL IX,1,1,260000,Orang/Shift,Hari Raya Natal,TNI,natal


In [15]:
df.to_csv("training_bko.csv")

In [18]:
df.to_csv("training_bko_final.csv", index=False)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pickle
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

# Encoding categorical variables
label_encoder_lokasi = LabelEncoder()
df['Lokasi'] = label_encoder_lokasi.fit_transform(df['Lokasi'])

label_encoder_event = LabelEncoder()
df['keyword'] = label_encoder_event.fit_transform(df['keyword'])

# Step 1: Prepare the data
X = df[['Hari', 'Lokasi', 'keyword']].values
y = df['Shift'].values

# Reshape data for LSTM: (samples, time steps, features)
# Here, we'll assume each sequence has 1 time step (since it's not a time series data in your example)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Step 2: Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Step 3: Train the model
model.fit(X, y, epochs=200, verbose=0)

# Step 4: Make predictions
y_pred = model.predict(X)

# Evaluate the model (optional since we only have training data here)
y_pred_rounded = np.round(y_pred).astype(int).flatten()
accuracy = accuracy_score(y, y_pred_rounded)

print(f"Predictions: {y_pred_rounded}")
print(f"True values: {y}")
print(f"Accuracy: {accuracy}")

c:\Users\gaudh\anaconda3\envs\SatriaData\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step
Predictions: [1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1
 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1
 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2
 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1
 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2
 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1
 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2
 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2
 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2
 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2
 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2]
True values: [1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1
 1 2 1 2 1 2 2 2 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 2 

In [56]:
model.save('model/predict/lstm_shift.h5')

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Save the model to a .tflite file
with open('model/predict/lstm_shift.tflite', 'wb') as f:
    f.write(model)

TypeError: a bytes-like object is required, not 'Sequential'